In [11]:
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import pickle

# import open bandit pipeline (obp)
import obp
from obp.policy import (
    IPWLearner, 
    QLearner,
    NNPolicyLearner, 
    Random
)

In [12]:
def UIR_to_train(input_file,output_file):
    df = pd.read_csv(input_file,header=None)[:1000]


    df.columns = ['userID','itemID','rating']
    df.shape



    #df_L = df.groupby('user_id')[['item_id','rating']].apply(lambda list: ','.join(map(str,list)).strip())
    df_L = df.groupby('userID')[['itemID','rating']].apply(agg_fnc).reset_index()

    y = np.array(df_L.index.to_list())
    X = np.array(df_L["userID"]).reshape(-1,1)
    # print(X,y)
    ipw_learner =ipw_train_learner(X,y,10)
    
    
    print(ipw_learner.predict(X).reshape(-1,df_L.shape[0]))
    
    with open(output_file, 'wb') as f:
        pickle.dump((ipw_learner,df_L), f)
    f.close()
    
    return df_L, ipw_learner

def ipw_train_learner(context,actions,max_iter=5000):
    
    # define NNPolicyLearner with IPW as its objective function
    ipw_rf = IPWLearner(
        n_actions=len(actions),
        base_classifier = RandomForest(n_estimators=1000,random_state=0).fit(context, actions)
        
        #base_classifier=RandomForest(
        #n_estimators=30, min_samples_leaf=10, random_state=12345
        #)
    )


    rewards = np.ones(len(context))

    # train NNPolicyLearner on the training set of logged bandit data
    ipw_rf.fit(
        context=context,
        action=actions,
        reward=rewards
    )
    
    return ipw_rf

In [13]:
def agg_fnc(x):
    d = []
    d.append(','.join(map(str,x["itemID"])))
    d.append(','.join(map(str,x["rating"])))
    return pd.Series(d,index=["L","L_rating"])

In [14]:
#demo01 uses re-ranking. This is the reason it was chose for demoOBP_FAIR. The results will be values that are reranked.
input_file = 'demo01-out-1.txt' #change the name from librec-auto (out-1.txt) to (demo02-out-1.txt)
output_file = 'IPW_OBP_demo01.pickle' #demo-2021 FAIR
df_L1, learner1 = UIR_to_train(input_file,output_file)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [ ]:
#Demo02 has the algorithm and no re-ranking. 
input_file = 'demo02-out-1.txt' #change the name from librec-auto (out-1.txt) to (demo02-out-1.txt)
output_file = 'IPW_OBP_demo02.pickle' #demo-2021 STANDARD
df_L2, learner2= UIR_to_train(input_file,output_file)